## Avaliação - Classificação Automática de Artigos Wikipedia

Nesta prática, iremos usar dados de 3.294 artigos da Wikipédia rotulados manualmente quanto a sua qualidade. 

Esses artigos passaram por uma avaliação pela comunidade de editores da Wikipedia. Tais editores classificaram esses artigos quanto a qualidade da seguinte forma: 

- **Artigo Destaque (FA)**: Os artigos atribuídos a esta classe são, de acordo com os avaliadores, os melhores artigos da Wikipédia.
- **Classe A (AC)**: os artigos da Classe A são considerados completos, mas com alguns problemas pendentes que precisam ser resolvidos para serem promovidos a Artigos em destaque.
- **Artigo Bons (GA)**: Bons Artigos são aqueles sem problemas de lacunas ou conteúdo excessivo. Essas são boas fontes de informação, embora outras enciclopédias possam fornecer um conteúdo melhor.
- **Classe B (BC)**: os artigos atribuídos a essa classe são considerados úteis para a maioria dos usuários, mas carecem de informações mais precisas.
- **Classe Inicial (ST)**: os artigos da Classe Inicial ainda estão incompletos, embora contenham referências e ponteiros para informações mais completas.
- **Artigos Rascunhos (SB)**: os artigos de toco são artigos de rascunho, com poucos parágrafos. Eles também têm poucas ou nenhumas citações.

Assim, [Dalip et. al. (2009)](https://dl.acm.org/citation.cfm?id=1555449) fizeram o preprocessamento desses artigos para serem extraídos indicadores de qualidades tais como: idade do artigo, tamanho, número de citações. Com tais indicadores e a classe de qualidade, foi possível realizar a predição automática de qualidade de artigos da Wikipédia.

Nesta prática, iremos fazer a previsão da qualidade usando os indicadores proposto por [Dalip et. al. (2009)](https://dl.acm.org/citation.cfm?id=1555449) e uma árvore de decisão.

Inicialmente, uso um DataFrame pandas e leia o arquivo wikipedia.csv e exiba os dados deste dataset. Coloque como o rótulo da linha o id do artigo (ou seja, no dataset, a coluna id será a index_col do DataFrame).

1. **Implementação do código de avaliação:** Primeiramente você deverá implementar as métricas de avaliação (da classe Resultado). O arquivo `resultado_tests.py` possui os testes unitários.  Veja abaixo como cada métrica, que é uma propriedade da classe (i.e. atributo calculado):
    - **mat_confusão**: Retorna a matriz de confusão correpondente. Será uma matriz em que o número de linhas e coluna é o valor numérico da maior classe na amostra.
    - **acurácia**: A partir da matriz de confusão, calcule a acurácia 
    - **precisao**: A partir da matriz de confusão, calcule a precisão por classe. Caso, a quantidade de instancias preditas para uma determinda classe for zero, então `precisao[c] = 0`. Nesses casos, você deverá [lançar um warning](https://docs.python.org/3.7/library/warnings.html) da classe `UndefinedMetricWarning` com uma mensagem que não havia instancias previstas para essa classe.
    - **revocacao**: De forma similar à `precisao`, calcula a revocação por meio da matriz de confusão. Caso o número de elementos dessa classe seja igual a zero, então a revocação para esta classe também é zero e também deverá ser retornado um warning `UndefinedMetricWarning` com essa informação
    - **f1_por_classe**: Retorna, para cada classe, o seu valor F1. Caso a soma da precisão e revocação dessa classe seja zero, deverá ser retornado zero.
    - **macro_f1**: Calcula a média do f1 por classe. O método `np.average` pode ajudar.
    
A matriz de confusão já está implementada, as demais possuem algo para você complementar. Se necessário, <a href="https://medium.com/thalus-ai/performance-metrics-for-classification-problems-in-machine-learning-part-i-b085d432082b">Explicações sobre precisão e revocação</a> Veja também nos slides



2. **Método eval da classe fold**: O método `eval` passará como parametro um método de aprendizado de máquina (por ex, uma instancia de [Árvore de Decisão](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)). Assim, usando esse parametro, você deverá criar o modelo de treino (usando as features de `df_treino` e a sua classe) e classificar os elementos de `df_data_to_predict`.

3. **Obtenção e divisão do Dataset:** Nesta prática, iremos trabalhar com 80% de treino e 20% de teste. Leia o dataset [`segment.csv`](segment.csv) e divida-o apropriadamente. Não esqueça que a amostra deve ser aleatória. Coloque como `random_state=1`

In [1]:
import pandas as pd

df = pd.read_csv("wikipedia.csv", ).drop(columns=["id"])

class_dict = {
    "'FA'":5,
    "'AC'":4,
    "'GA'":3,
    "'BC'":2,
    "'ST'":1,
    "'SB'":0
}

df["realClass"] = df["realClass"].apply(lambda x: class_dict[x])

df_treino = df.sample(frac=0.8, random_state=1)
df_data_to_predict = df.drop(df_treino.index)

4. **Criação do modelo e avaliação** dos modelos: agora, você deverá avaliar 4 modelos de aprendizado de máquina nessa tarefa. Use os métodos SVM com kernel linear e RBF, KNN e árvore de decisão. Deixe os parametros padrão de cada algoritmo. Por meio do método eval da classe Fold, avalie o método. Apresente a matriz de confusão resultante de cada um desses métodos além das métricas macro f1, precisão e revocação por classe. Também verifique  quão sensível são os resultados se mudarmos o parametro `random_seed`. Você pode criar mais de um bloco de código/texto para isso organizando da forma que julgar melhor. 

    <ul>
        <li><a href="https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC">SVM</a></li>
        <li><a href="https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html">KNN</a></li>
    <li><a href="https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html">Árvore de decisão</a></li>
    </ul>


In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from resultado import Fold, Resultado

col_classe = "realClass"
fold = Fold(df_treino,df_data_to_predict,col_classe)

svm_method = SVC(gamma='auto')
knn_method = KNeighborsClassifier(n_neighbors=3)
tree_method = DecisionTreeClassifier(random_state=1)


svm_result = fold.eval(svm_method)
knn_result = fold.eval(knn_method)
tree_result = fold.eval(tree_method)

In [3]:
pd.DataFrame([
                [svm_result.acuracia, svm_result.macro_f1],
                [knn_result.acuracia, knn_result.macro_f1],
                [tree_result.acuracia, tree_result.macro_f1]
             ],
             columns = ['Acurácia', 'Macro F1'], 
             index = ['SVM', 'KNN', 'Tree'])


positives
[False False False False False  True]
positives
[ True  True  True  True  True  True]
positives
[ True  True  True  True  True  True]


/home/ana/Documents/tcc-web-crawler/wikipedia_dataset_hasan/resultado.py:82: UndefinedMetricWarning: divisão por zero no cálculo da precisão
  warnings.warn("divisão por zero no cálculo da precisão", UndefinedMetricWarning)


,Acurácia,Macro F1
SVM,0.154780,0.268068
KNN,0.444613,0.429839
Tree,0.474962,0.472755


In [4]:
svm_result.mat_confusao

array([[  0.,   0.,   0.,   0.,   0., 112.],
       [  0.,   0.,   0.,   0.,   0., 116.],
       [  0.,   0.,   0.,   0.,   0., 102.],
       [  0.,   0.,   0.,   0.,   0., 111.],
       [  0.,   0.,   0.,   0.,   0., 116.],
       [  0.,   0.,   0.,   0.,   0., 102.]])

In [5]:
pd.DataFrame([
                svm_result.precisao,
                knn_result.precisao,
                tree_result.precisao
             ],
             columns = ['SB', 'ST', 'BC', 'GA', 'AC', 'FA'],
             index = ['SVM', 'KNN', 'Tree'])

,SB,ST,BC,GA,AC,FA
SVM,0.000000,0.000000,0.000000,0.00000,0.000000,0.154780
KNN,0.765625,0.442857,0.309735,0.32800,0.350877,0.385417
Tree,0.724138,0.518182,0.385965,0.47191,0.324074,0.418033


In [6]:
pd.DataFrame([
                svm_result.revocacao,
                knn_result.revocacao,
                tree_result.revocacao
             ],
             columns = ['SB', 'ST', 'BC', 'GA', 'AC', 'FA'],
             index = ['SVM', 'KNN', 'Tree'])

,SB,ST,BC,GA,AC,FA
SVM,0.000,0.000000,0.000000,0.000000,0.000000,1.000000
KNN,0.875,0.534483,0.343137,0.369369,0.172414,0.362745
Tree,0.750,0.491379,0.431373,0.378378,0.301724,0.500000


5. **Conclusões:** Escreva um texto com uma análise e conclusão dos resultados, por exemplo: quais são as classes mais dificieis/fácieis de prever? Quais se confundem mais? Qual é o melhor método de classificação?

O melhor método para a classificação foi a árvore de decisão que obteve tanto acurácia quanto macro F1 mais altos. 
No método de classificação de árvore binária as classes brikface, sky e grass obtiveram revocacao de 1, indicando que todos as imagens dessas classes foram classificadas corretamente. A precisão das classes para este método também foram ótimos, todos acima de 0.86, indicando que a predição foi bem acertiva.

Embora o SVM tenha contado com precisão 1 para 6 classes, que significa que todas as imagens classificadas dessas classes foram classificadas corretamente, a precisão da classe cement foi muito baixa. Isso significa que muitas imagens de outras classes foram preditas erroneamente como sendo cement. Isso fica bastante claro nas métricas de revocação em que apenas a classe cement atinge 1 enquanto as outras classes estão abaixo de 0.6.